In [1]:
# from ranking import ranking_pipeline
from preprocessing import load_dataset, build_fitness_matrix
from collections import namedtuple
from lightgbm import LGBMRanker
from sklearn.model_selection import train_test_split
from findhr.preprocess.example_mappings import MatchBinary, MatchOrdinal, MatchFeatureInclusion, MatchFeatureSet
from findhr.preprocess.mapping import AttachMetadata, DetachMetadata, DerivedColumn
from findhr.xai.counterfactual import dice_ml
from sklearn.pipeline import Pipeline
from metadata import md_CDS_JDS_ADS
from MACRO import MacroVariables
import pandas as pd
import ast

In [2]:
TEST_SIZE = 0.2
# TRAIN_SIZE = 0.8
VAL_SIZE = 0.25  # 0.25 x 0.8 = 0.2

In [3]:
## Define the Preprocessing pipeline

In [4]:
# Helper function to convert columns of lists of values
def convert_cols(x):
    if isinstance(x, int) or isinstance(x, float) or isinstance(x, list):
        return x
    try:
        x = ast.literal_eval(x)
    finally:
        return x

# Helper function to convert columns of lists of values
def rank2relevance(df, top_k, col_rank):
    return top_k + 1 - df[col_rank].values.ravel()


In [5]:
def load_dataset(fair_data=True):
    # Read dataset
    df_JDS = pd.read_csv(MacroVariables.FILEPATH_JOB_OFFERS,  # converters for columns of lists of values
                         converters={c:convert_cols for c in ["Age_j", "Competences_j", "Knowledge_j", "Languages_j"]})
    df_CDS = pd.read_csv(MacroVariables.FILEPATH_CURRICULA,  # converters for columns of lists of values
                         converters={c:convert_cols for c in ["Age_c", "Experience_c", "Competences_c", "Knowledge_c", "Languages_c"]})
    df_ADS_FAIR = pd.read_csv(MacroVariables.FILEPATH_ADS_FAIR)
    df_ADS_UNFAIR = pd.read_csv(MacroVariables.FILEPATH_ADS_UNFAIR)

    # Define subsets of columns
    cols_id = ['qId', 'kId']
    # Define the subset of columns of the HUDD dataset describing the candidate,
    # which are used in the preprocessing+prediction pipeline
    cols_c = ['Education_c', 'Age_c', 'Gender_c', 'Contract_c',
              'Nationality_c', 'Competences_c', 'Knowledge_c', 'Languages_c',
              'Experience_c']
    cols_j = ['Education_j', 'Age_j', 'Gender_j',  'Contract_j', 'Nationality_j', 'Competences_j',
              'Knowledge_j', 'Languages_j', 'Experience_j']
    cols_pred_preprocess = cols_c + cols_j
    cols_not_for_pred = ['Occupation_c', 'Occupation_j']
    cols_sensitive = ['Gender_c']
    col_target = ['score']

    if fair_data:
        # Merge CDS and JDS through ADS in a single dataframe
        df_CDS_JDS = pd.merge(df_ADS_FAIR, df_JDS, on='qId')
    else:
        # Merge CDS and JDS through ADS in a single dataframe
        df_CDS_JDS = pd.merge(df_ADS_UNFAIR, df_JDS, on='qId')

    df_CDS_JDS = pd.merge(df_CDS, df_CDS_JDS, on='kId')
    df_CDS_JDS = df_CDS_JDS[cols_id + [col for col in df_CDS_JDS if col not in cols_id+col_target] + col_target ]

    return df_CDS_JDS, {'cols_pred_preprocess': cols_pred_preprocess,
                        'cols_sensitive': cols_sensitive,
                        'cols_id': cols_id,
                        'cols_not_for_pred': cols_not_for_pred,
                        'col_target': col_target}

def build_matching_functions():
    # Matching functions for pairs of job-candidate features
    maps_matching = {
         # MatchBinary: 1 = job value = candidate value OR job value is 'Any' OR candidate value is 'Any', 0 = otherwise
        # (('qId',), ('qId',)): IdentityMapping(),
        # (('kId',), ('kId',)): IdentityMapping(),
        # (('rank',), ('rank',)): IdentityMapping(),
        (('Contract_j', 'Contract_c'), ('fitness_Contract',)): MatchBinary(),
        (('Gender_j', 'Gender_c'), ('fitness_Gender',)): MatchBinary(),
        (('Nationality_j', 'Nationality_c'), ('fitness_Nationality',)): MatchBinary(),

         # MatchOrdinal: 1 = job value >= candidate OR job value is 'Any', 0 = otherwise
        (('Education_j', 'Education_c'), ('fitness_Education',)): MatchOrdinal(),
        (('Experience_j', 'Experience_c'), ('fitness_Experience',)): MatchOrdinal(),

         # MatchFeatureInclusion: 1 = candidate value in (job value(0,), >= job value(1,)) OR job value is 'Any', 0 = otherwise
        # (('Age_j', 'Age_c'), ('fitness_Age',)): MatchFeatureInclusion(),

         # MatchFeatureSet: 1 = fraction of job value that appear in candidate value
        (('Languages_j', 'Languages_c'), ('fitness_Languages',)): MatchFeatureSet(),
        (('Competences_j', 'Competences_c'), ('fitness_Competences',)): MatchFeatureSet(),
        (('Knowledge_j', 'Knowledge_c'), ('fitness_Knowledge',)): MatchFeatureSet()
    }
    return maps_matching


def build_fitness_matrix(df_CDS_JDS, cols_dict, fair_data=True):
    maps_matching = build_matching_functions()

    # Calculation as fit-transform preprocessing
    pipeline_fitness = Pipeline(steps=[
        ("init", AttachMetadata(md_CDS_JDS_ADS)),
        ("matching", DerivedColumn(maps_matching)),
        ("end", DetachMetadata())
    ])

    pipeline_fitness.fit(X=df_CDS_JDS)
    fitness_matrix = pipeline_fitness.transform(X=df_CDS_JDS)
    df_fitness_mat = fitness_matrix.copy(deep=True)
    columns_keep = cols_dict['cols_id'] + \
                   [col for col in fitness_matrix if
                    col.startswith('fitness_')] + cols_dict['cols_sensitive'] + cols_dict['col_target']

    df_fitness_mat = df_fitness_mat[columns_keep]

    # From scores, we can learn regressors; or we can produce ranks, and learn ranking models
    df_fitness_mat['rank'] = df_fitness_mat.groupby("qId")['score'].rank('dense', ascending=False)
    df_fitness_mat['rank'] = df_fitness_mat['rank'].apply(lambda x: x if x <= MacroVariables.TOP_K else MacroVariables.TOP_K + 1)

    return pipeline_fitness, df_fitness_mat

## Define the ranking model

In [6]:
def data_split_FEDD(df_fitness_mat):
    all_jobs = df_fitness_mat['qId'].unique()
    train_jobs, test_jobs = train_test_split(all_jobs, test_size=0.2, random_state=42, shuffle=False)
    train_jobs, val_jobs = train_test_split(train_jobs, test_size=0.25, random_state=42, shuffle=False)

    # Build train, test and validation sets, ensuring they are sorted by qId, kId
    df_train = df_fitness_mat[df_fitness_mat['qId'].isin(train_jobs)].sort_values(["qId", "kId"])
    df_val = df_fitness_mat[df_fitness_mat['qId'].isin(val_jobs)].sort_values(["qId", "kId"])
    df_test = df_fitness_mat[df_fitness_mat['qId'].isin(test_jobs)].sort_values(["qId", "kId"])

    return df_train, df_val, df_test


def init(df_fitness_mat):
    df_train, df_val, df_test = data_split_FEDD(df_fitness_mat)
    # Define subsets of columns
    cols_id = ['qId', 'kId']  # ids
    cols_pred = [  # predictive
        'fitness_Contract',
        'fitness_Nationality',
        'fitness_Education',
        'fitness_Experience',
        # 'fitness_Age',
        'fitness_Gender',
        'fitness_Languages',
        'fitness_Competences',
        'fitness_Knowledge']
    cols_sensitive = ['Gender_c']  # sensitive attribute(s)
    col_target = 'score'  # target value for ranking
    col_rank = 'rank'  # rank value for ranking

    cols_dict_FEDD = {'cols_id': cols_id,
                      'cols_pred': cols_pred,
                      'cols_sensitive': cols_sensitive,
                      'col_target': col_target,
                      'col_rank': col_rank}
    # Define the ranking model
    ranker = LGBMRanker(
        objective="lambdarank",
        class_weight="balanced",
        boosting_type="gbdt",
        importance_type="gain",
        learning_rate=0.1,
        n_estimators=100,
        force_row_wise=True,
        n_jobs=-1,  # max parallelism
        verbose=-1  # no verbosity
    )
    return ranker, df_train, df_val, df_test, cols_dict_FEDD


def train(ranker, df_train, df_val, cols_dict):
    df_train_counts = df_train.groupby("qId")["qId"].count().to_numpy()
    df_val_counts = df_val.groupby("qId")["qId"].count().to_numpy()

    # Fitting ranker:
    ranker.fit(
        X=df_train[cols_dict['cols_pred']],
        # LightGBM relevance is the higher the better
        y=rank2relevance(df_train, MacroVariables.TOP_K, cols_dict['col_rank']),
        group = df_train_counts,
        eval_at = [MacroVariables.TOP_K],
        # LightGBM relevance is the higher the better
        eval_set =[(df_val[cols_dict['cols_pred']], rank2relevance(df_val, MacroVariables.TOP_K, cols_dict['col_rank']))],
        eval_group =[df_val_counts]
    )

    return ranker


def evaluate(ranker, df_test, cols_dict):
    df_test_counts = df_test.groupby("qId")["qId"].count().to_numpy()
    # Predicting ranker:
    df_test['lambda'] = ranker.predict(df_test[cols_dict['cols_pred']])
    df_test['pred_rank'] = df_test.groupby("qId")['lambda'].rank('dense', ascending=False)
    df_test['pred_rank'] = df_test['pred_rank'].apply(lambda x: x if x <= MacroVariables.TOP_K else MacroVariables.TOP_K + 1)

    return df_test


def ranking_pipeline(df_fitness_mat):
    ranker, df_train, df_val, df_test, cols_dict_FEDD = init(df_fitness_mat)
    ranker = train(ranker, df_train, df_val, cols_dict_FEDD)
    df_test = evaluate(ranker, df_test, cols_dict_FEDD)
    return ranker, df_test, cols_dict_FEDD

## Prepare the data for the counterfactual explanation

In [7]:
def define_cols_dict():
    outcome_name_col = 'lambda'  # 'pred_rank'
    continuous_features = ['fitness_Languages', 'fitness_Competences',
                           'fitness_Knowledge']  # ['Age_c', 'Experience_c'],
    categorical_features = ['fitness_Contract', 'fitness_Nationality', 'fitness_Education', 'fitness_Experience',
                            # 'fitness_Age',
                            'fitness_Gender']
    cols_pred = continuous_features + categorical_features
    return {'outcome_name_col': outcome_name_col, 'continuous_features': continuous_features,
            'categorical_features': categorical_features, 'cols_pred': cols_pred}


def extract_explicand_data_cf(job_id, candidate_position, df_test, ranker, cols_dict_FEDD, df_CDS_JDS):
    # df_qId contains the data for the job qId
    df_qId_FEDD = df_test[df_test['qId'] == job_id]
    df_qId_FEDD['lambda'] = ranker.predict(df_qId_FEDD[cols_dict_FEDD['cols_pred']])
    df_qId_FEDD['pred_rank'] = df_qId_FEDD.groupby("qId")['lambda'].rank('dense', ascending=False)

    exp_c_pred_rank = candidate_position

    # Extract the explicand candidate kId
    exp_c_kId = df_qId_FEDD.loc[df_qId_FEDD['pred_rank'] == exp_c_pred_rank, 'kId'].iloc[0]

    # Isolate the candidates' profiles applying for the job qId
    df_qId_HUDD = df_CDS_JDS[df_CDS_JDS['qId'] == job_id]

    # Isolate the explicand candidate profile
    exp_c_profile = df_CDS_JDS[df_CDS_JDS['kId'] == exp_c_kId]

    exp_c = {'kId': exp_c_kId, 'profile': exp_c_profile}

    cols_dict = define_cols_dict()

    return df_qId_FEDD, df_qId_HUDD, exp_c, cols_dict


def prepare_data_cf(df_qId_FEDD, cols_dict):

    # Convert data types
    df_qId_FEDD_pre = df_qId_FEDD[cols_dict['categorical_features']].astype('int').copy(deep=True)
    df_qId_FEDD_pre[cols_dict['continuous_features']] = df_qId_FEDD[cols_dict['continuous_features']].astype('float').copy(deep=True)
    df_qId_FEDD_pre[cols_dict['outcome_name_col']] = df_qId_FEDD[cols_dict['outcome_name_col']].copy(deep=True)
    feature_dtypes = {col: df_qId_FEDD_pre[col].dtype for col in df_qId_FEDD_pre[cols_dict['cols_pred']].columns}

    return df_qId_FEDD_pre, feature_dtypes


def define_target(args, df_qId_FEDD):
    # 'in_top_k' or 'out_top_k' depending on the candidate position
    explicand_class = 'in_top_k' if args.candidate_position <= MacroVariables.TOP_K else 'out_top_k'

    # target rank for counterfactual explanation
    if args.target_rank:
        tgt_cf_rank = args.target_rank
        tgt_cf_score = df_qId_FEDD[df_qId_FEDD['pred_rank'] == tgt_cf_rank]['score'].iloc[0]
        tgt_cf_candidate = df_qId_FEDD[df_qId_FEDD['pred_rank'] == tgt_cf_rank]

    elif args.target_score:
        tgt_cf_rank = None
        tgt_cf_score = args.target_score
        tgt_cf_candidate = None
    else:
        raise ValueError('Either target rank or target score must be provided')

    return explicand_class, tgt_cf_rank, tgt_cf_score, tgt_cf_candidate


def define_explainer_FEDD(ranker, df_qId_FEDD_pre, cols_dict_cf, feature_dtypes, explanation_method):
    data_dice = dice_ml.Data(dataframe=df_qId_FEDD_pre[cols_dict_cf['cols_pred'] + [cols_dict_cf['outcome_name_col']]],
                             continuous_features=cols_dict_cf['continuous_features'],
                             categorical_features=cols_dict_cf['categorical_features'],
                             outcome_name=cols_dict_cf['outcome_name_col'])

    kwargs = {'top_k': MacroVariables.TOP_K, 'features_dtype': feature_dtypes}

    model_dice = dice_ml.Model(model=ranker,
                               backend={'explainer': 'dice_xgboost.DiceGenetic',
                                        'model': "lgbmranker_model.LGBMRankerModel"},
                               model_type="regressor",
                               # model_type="classifier",
                               kw_args=kwargs)

    explainer = dice_ml.Dice(data_dice, model_dice, method=explanation_method)

    return explainer, data_dice, model_dice


def get_explanations_FEDD(df_qId_FEDD, exp_c, cols_dict_cf, explainer):

    c_th_lambda = df_qId_FEDD[df_qId_FEDD['pred_rank'] == MacroVariables.TOP_K].iloc[0]['lambda']
    explanations = explainer.generate_counterfactuals(exp_c['profile'][cols_dict_cf['cols_pred']],
                                                      total_CFs=10,
                                                      desired_range=[c_th_lambda, 100],
                                                      # desired_class="opposite",
                                                      verbose=True)
    return explanations

In [10]:

# The job id for which the counterfactual explanation is to be generated'
# Valid values 160-199
job_id = 163

# The position of the candidate in the ranked list
candidate_position = 9

# Alternative ways to define the target of the explanation
target_rank = MacroVariables.TOP_K
target_score = 0.9

explanation_method = 'genetic'


In [11]:
args = namedtuple('Args', ['target_rank', 'target_score', 'target_candidate'])

args.target_rank = target_rank
args.target_score = target_score
args.candidate_position = candidate_position

df_CDS_JDS, cols_dict_HUDD = load_dataset(fair_data=MacroVariables.FAIR_DATA)
pipeline_fitness, df_fitness_mat = build_fitness_matrix(df_CDS_JDS, cols_dict_HUDD, fair_data=MacroVariables.FAIR_DATA)
ranker, df_test, cols_dict_FEDD = ranking_pipeline(df_fitness_mat)
from cf_exp_FEDD import extract_explicand_data_cf, prepare_data_cf, define_target, define_explainer_FEDD, get_explanations_FEDD

df_qId_FEDD, df_qId_HUDD, exp_c, cols_dict_cf = extract_explicand_data_cf(job_id, candidate_position, df_test, ranker, cols_dict_FEDD, df_CDS_JDS)
# Convert data types
df_qId_FEDD_pre, feature_dtypes = prepare_data_cf(df_qId_FEDD, cols_dict_cf)

explicand_class, tgt_cf_rank, tgt_cf_score, tgt_cf_candidate = define_target(args, df_qId_FEDD)

explainer, data_dice, model_dice = define_explainer_FEDD(ranker, df_qId_FEDD_pre, cols_dict_cf, feature_dtypes, explanation_method)
print('Explanations for the counterfactuals:')


/Users/mastropi/postdoc_works/xai_findhr/code/DiCE_findhr/script/cf_exp_FEDD.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qId_FEDD['lambda'] = ranker.predict(df_qId_FEDD[cols_dict_FEDD['cols_pred']])
/Users/mastropi/postdoc_works/xai_findhr/code/DiCE_findhr/script/cf_exp_FEDD.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qId_FEDD['pred_rank'] = df_qId_FEDD.groupby("qId")['lambda'].rank('dense', ascending=False)
/Users/mastropi/postdoc_works/xai_findhr/code/DiCE_findhr/script/cf_exp_FEDD

Explanations for the counterfactuals:


  0%|          | 0/1 [00:00<?, ?it/s]

Initializing initial parameters to the genetic algorithm...


/Users/mastropi/postdoc_works/xai_findhr/code/DiCE_findhr/dice_ml/explainer_interfaces/dice_genetic.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  feature_weights_list.append(round(1 / self.feature_range[feature].max(), 2))
100%|██████████| 1/1 [00:00<00:00,  1.47it/s]

Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Query instance (original outcome : -0.09421838819980621)


,fitness_Languages,fitness_Competences,fitness_Knowledge,fitness_Contract,fitness_Nationality,fitness_Education,fitness_Experience,fitness_Gender,lambda
0,0.0,1.0,0.333333,1,0,1,1,1,-0.094218



Diverse Counterfactual set (new outcome: [-2.631657100755981, 100])


,fitness_Languages,fitness_Competences,fitness_Knowledge,fitness_Contract,fitness_Nationality,fitness_Education,fitness_Experience,fitness_Gender,lambda
0,0.0,1.0,0.3,1,0,1,1,1,-0.094218
0,0.0,1.0,0.4,1,0,1,1,1,-0.094218
0,0.0,1.0,0.2,1,0,1,1,1,-0.094218
0,0.0,1.0,0.6,1,0,1,1,1,-0.094218
0,0.0,1.0,0.3,1,0,0,1,1,-0.094218
0,0.0,1.0,0.3,1,1,1,1,1,-0.094218
0,0.0,1.0,0.4,1,0,0,1,1,-0.094218
0,0.0,1.0,0.4,1,1,1,1,1,-0.094218
0,0.0,1.0,0.0,1,0,1,1,1,0.483927
0,0.0,0.9,0.3,1,0,1,1,1,-0.094218


None


In [12]:
explanations_FEDD = get_explanations_FEDD(df_qId_FEDD, exp_c, cols_dict_cf, explainer)



  0%|          | 0/1 [00:00<?, ?it/s]

Initializing initial parameters to the genetic algorithm...


100%|██████████| 1/1 [00:00<00:00,  1.75it/s]

Initialization complete! Generating counterfactuals...
Diverse Counterfactuals found! total time taken: 00 min 00 sec
Query instance (original outcome : -0.09421838819980621)


,fitness_Languages,fitness_Competences,fitness_Knowledge,fitness_Contract,fitness_Nationality,fitness_Education,fitness_Experience,fitness_Gender,lambda
0,0.0,1.0,0.333333,1,0,1,1,1,-0.094218



Diverse Counterfactual set (new outcome: [-2.631657100755981, 100])


,fitness_Languages,fitness_Competences,fitness_Knowledge,fitness_Contract,fitness_Nationality,fitness_Education,fitness_Experience,fitness_Gender,lambda
0,0.0,1.0,0.3,1,0,1,1,1,-0.094218
0,0.0,1.0,0.4,1,0,1,1,1,-0.094218
0,0.0,1.0,0.2,1,0,1,1,1,-0.094218
0,0.0,1.0,0.6,1,0,1,1,1,-0.094218
0,0.0,1.0,0.3,1,0,0,1,1,-0.094218
0,0.0,1.0,0.4,1,0,0,1,1,-0.094218
0,0.0,1.0,0.0,1,0,1,1,1,0.483927
0,0.0,0.9,0.3,1,0,1,1,1,-0.094218
0,0.0,1.0,0.2,1,0,0,1,1,-0.094218
0,0.0,1.0,0.5,1,0,0,1,1,-0.094218


None


In [13]:
print(explanations_FEDD.visualize_as_dataframe())

explanations_FEDD.cf_examples_list[0].final_cfs_df.to_csv('final_cfs_df_FEDD.csv')

Query instance (original outcome : -0.09421838819980621)


,fitness_Languages,fitness_Competences,fitness_Knowledge,fitness_Contract,fitness_Nationality,fitness_Education,fitness_Experience,fitness_Gender,lambda
0,0.0,1.0,0.333333,1,0,1,1,1,-0.094218



Diverse Counterfactual set (new outcome: [-2.631657100755981, 100])


,fitness_Languages,fitness_Competences,fitness_Knowledge,fitness_Contract,fitness_Nationality,fitness_Education,fitness_Experience,fitness_Gender,lambda
0,0.0,1.0,0.3,1,0,1,1,1,-0.094218
0,0.0,1.0,0.4,1,0,1,1,1,-0.094218
0,0.0,1.0,0.2,1,0,1,1,1,-0.094218
0,0.0,1.0,0.6,1,0,1,1,1,-0.094218
0,0.0,1.0,0.3,1,0,0,1,1,-0.094218
0,0.0,1.0,0.4,1,0,0,1,1,-0.094218
0,0.0,1.0,0.0,1,0,1,1,1,0.483927
0,0.0,0.9,0.3,1,0,1,1,1,-0.094218
0,0.0,1.0,0.2,1,0,0,1,1,-0.094218
0,0.0,1.0,0.5,1,0,0,1,1,-0.094218


None
